In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import json
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Import Data

In [70]:
df_covid = pd.read_csv("covid19_news.csv")

In [71]:
len(df_covid)

4619

In [72]:
df_covid.sample(10)

,doc_content,doc_date,doc_id,doc_source,doc_title,doc_url
764,"ALBANY, N.Y. (AP) — The state University at Al...",02/20/2020,1021697,US News,Students' Coronavirus-Themed Party Stirs Anger...,https://www.usnews.com/news/best-states/new-yo...
4031,"By KIM CHANDLER, Associated Press\nMONTGOMERY,...",03/25/2020,1039624,US News,Alabama Unemployment Claims Skyrocket Amid Pan...,https://www.usnews.com/news/best-states/alabam...
471,closeVideoFox News Flash top headlines for Feb...,02/06/2020,1009458,Fox,Airline passenger forces WestJet plane back to...,https://www.foxnews.com/travel/airline-passeng...
468,(This Feb. 8 story has been corrected to remov...,02/06/2020,1009323,US News,American Dies of Coronavirus in China; Five Br...,https://www.usnews.com/news/world/articles/202...
749,BEIJING/SHANGHAI (Reuters) - China plans to ta...,02/19/2020,1020395,US News,China to Take Over HNA as Coronavirus Hits Bus...,https://www.usnews.com/news/world/articles/202...
194,\nSen. Tom Cotton (R-AR) is calling on the Tru...,01/28/2020,1001651,Breitbart,Tom Cotton Calls for Targeted Travel Ban on Ch...,http://feedproxy.google.com/~r/breitbart/~3/Is...
2529,"By ANDREW MELDRUM, Associated Press\nJOHANNESB...",03/14/2020,1035719,US News,"Virus Cases Spread Across Africa, Nations Prep...",https://www.usnews.com/news/world/articles/202...
289,(Reuters) - The coronavirus epidemic continued...,01/31/2020,1004789,US News,Coronavirus Forces Postponement of Badminton O...,https://www.usnews.com/news/world/articles/202...
1297,"By JOHN LEICESTER, Associated Press\nPARIS (AP...",03/01/2020,1028575,US News,Virus Fears Close Down France's Famed Louvre M...,https://www.usnews.com/news/business/articles/...
3186,closeVideoDr. Anthony Fauci says it will be we...,03/18/2020,1037221,Fox,Fauci says it'll be weeks before we see the ef...,https://www.foxnews.com/media/dr-fauci-says-it...


In [63]:
df_covid.doc_content[4280]

"\nBrazilian President Jair Bolsonaro once again blasted the global “hysteria” over the Chinese coronavirus in a public address Tuesday, urging his people to get back to normal life and work as soon as they possibly can.\nIn an address to the nation, Bolsonaro called on local authorities such as mayors and state governors to start rolling back lockdown measures that have shut down schools and brought many of the country’s big cities to complete standstill.\n\n\n“Our lives have to go on. Jobs must be kept … we must, yes, get back to normal,” Bolsonaro said. “A small number of state and municipal authorities must abandon their scorched-earth ideas: the banning of public transport, the closing of commerce, and mass confinement.”\n\n“What is happening around the world has shown that the at-risk group are those aged 60 or over. So why close schools?” he continued. “90 percent of us will show no symptoms if we are infected.”\n\nUnable to take to the streets due to lockdown, opponents of Bols

In [248]:
df_covid.doc_id[2509]

1035069

## doc content preprocessing

### fox news

In [94]:
dt_fox = df_covid[df_covid.doc_source == 'Fox']

In [93]:
len(dt_fox)

0

In [77]:
dt_fox.head()

,doc_content,doc_date,doc_id,doc_source,doc_title,doc_url
14,BEIJING -- China reported Monday a sharp rise ...,01/20/2020,993253,Fox,China counts sharp rise in coronavirus cases d...,https://www.foxnews.com/health/china-counts-sh...
43,closeVideoSen. Tom Cotton links coronavirus sp...,01/22/2020,995686,Fox,Tom Cotton: China needs to answer for 'malfeas...,https://www.foxnews.com/media/tom-cotton-coron...
47,closeVideoTrump took swipe at Democratic candi...,01/22/2020,995366,Fox,Trump says US has plan to contain coronavirus,https://www.foxnews.com/health/trump-says-us-h...
54,closeVideoChina expands quarantine measures am...,01/23/2020,996817,Fox,Who are the coronavirus victims? What to know ...,https://www.foxnews.com/health/who-are-the-cor...
56,closeVideoWorld Health Organization officials ...,01/23/2020,996824,Fox,WHO says China's coronavirus 'too early' for e...,https://www.foxnews.com/health/who-china-coron...


In [78]:
index = list(dt_fox.doc_content.index.values)
index[:10]

[14, 43, 47, 54, 56, 67, 96, 115, 148, 150]

In [79]:
#dt_fox.doc_content[index[329]]

In [80]:
K = []

for i, d in enumerate(index):
    #print(i,d)
    text = df_covid.doc_content[d]
    para_lst = text.split('\n      ')
    
    para_lst = pd.DataFrame(para_lst)
    para_lst.columns = ['paragraph_str']
    er = df_covid.doc_id[i]
    para_lst = pd.concat([para_lst], keys=[er], names=['doc_id'])
    K.append(pd.DataFrame())
    
    K[i] = para_lst.paragraph_str.apply(lambda x: pd.Series(x.split('.'))).stack()
    K[i] = K[i].to_frame().rename(columns={0:'sentence_str'})
    
    K[i].index.names = ['doc_id','paragraph_num','sentence_num']
    K[i] = K[i].sentence_str.apply(lambda x: pd.Series(x.split())).stack()
    
    K[i] = K[i].to_frame().rename(columns={0:'token_str'})
    K[i].index.names = ['doc_id','paragraph_num','sentence_num','token_num']
    
    K[i]['term_str'] = K[i].token_str.replace('\W+', '', regex=True).str.lower()

#print (K[13], K[7])

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [81]:
K[2]

token_str         term_str
doc_id paragraph_num sentence_num token_num                                  
987316 0             0            0          closeVideoTrump  closevideotrump
                                  1                     took             took
                                  2                    swipe            swipe
                                  3                       at               at
                                  4               Democratic       democratic
...                                                      ...              ...
                     15           17                     and              and
                                  18                 Taiwan,           taiwan
                                  19               according        according
                                  20                      to               to
                                  21                    CNBC             cnbc

[267 rows x 2 columns]

In [82]:
fox = pd.concat(K,axis = 0)

In [83]:
fox

token_str  term_str
doc_id  paragraph_num sentence_num token_num                    
983556  0             0            0           BEIJING   beijing
                                   1                --          
                                   2             China     china
                                   3          reported  reported
                                   4            Monday    monday
...                                                ...       ...
1005471 2             73           9               her       her
                                   10          website   website
                                   11               at        at
                                   12          Komando   komando
                      74           0               com       com

[183327 rows x 2 columns]

### US news and CNN

In [123]:
dt_us_cnn_bre = df_covid[df_covid.doc_source != 'Fox' ]

In [124]:
len(dt_us_cnn_bre)

4289

In [127]:
index = list(dt_us_cnn_bre.doc_content.index.values)
#index[:30]

In [128]:
K = []

for i, d in enumerate(index):
    #print(i,d)
    text = df_covid.doc_content[d]
    para_lst = text.split('\n')
    
    para_lst = pd.DataFrame(para_lst)
    para_lst.columns = ['paragraph_str']
    er = df_covid.doc_id[i]
    para_lst = pd.concat([para_lst], keys=[er], names=['doc_id'])
    K.append(pd.DataFrame())
    
    K[i] = para_lst.paragraph_str.apply(lambda x: pd.Series(x.split('.'))).stack()
    K[i] = K[i].to_frame().rename(columns={0:'sentence_str'})
    
    K[i].index.names = ['doc_id','paragraph_num','sentence_num']
    K[i] = K[i].sentence_str.apply(lambda x: pd.Series(x.split())).stack()
    
    K[i] = K[i].to_frame().rename(columns={0:'token_str'})
    K[i].index.names = ['doc_id','paragraph_num','sentence_num','token_num']
    
    K[i]['term_str'] = K[i].token_str.replace('\W+', '', regex=True).str.lower()

#print (K[13], K[7])

/apps/software/standard/compiler/gcc/7.1.0/jupyter_conda/2019.10-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [129]:
us_cnn_bre = pd.concat(K,axis = 0)

In [131]:
us_cnn_bre.sample(20)

,,,,token_str,term_str
doc_id,paragraph_num,sentence_num,token_num,,
1030603,16,0,8,coronavirus,coronavirus
1003948,9,0,9,all,all
1037954,16,2,2,also,also
1033581,2,1,11,"duty,",duty
1039958,0,11,4,never,never
1039177,130,0,24,8,8
992548,3,0,21,"epidemic,",epidemic
1032992,9,1,6,few,few
1016604,15,0,6,positive,positive


In [132]:
news = [fox, us_cnn_bre]

In [133]:
total = pd.concat(news,axis = 0)

In [ ]:
total.sample()